In [1]:
!pip install -q gdown  # Install gdown if not already installed
import gdown

file_id = "1QS04Pm94JFqt1XjzxcpLrOD4Xb8gOBMD"  # Replace with your actual file ID
output = "Plant.zip"  # Change this to your desired file name

gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1QS04Pm94JFqt1XjzxcpLrOD4Xb8gOBMD
From (redirected): https://drive.google.com/uc?id=1QS04Pm94JFqt1XjzxcpLrOD4Xb8gOBMD&confirm=t&uuid=25d6aee9-6b54-4d45-ae20-3291f720d91e
To: /kaggle/working/Plant.zip
100%|██████████| 79.3M/79.3M [00:01<00:00, 59.1MB/s]


'Plant.zip'

In [2]:
import zipfile

with zipfile.ZipFile("/kaggle/working/Plant.zip", "r") as zip_ref:
    zip_ref.extractall("PlantDoc")  # Extract to 'data' folder

# Check extracted files
import os
print(os.listdir("PlantDoc"))

['data.yaml', 'test', 'README.dataset.txt', 'README.roboflow.txt', 'train']


In [3]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 15.7 MB/s eta 0:00:00


In [4]:
!pip install -q -U langchain langchain-community openai sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.5/474.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 24.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.


In [5]:
!pip install -q huggingface_hub googletrans transformers

In [6]:
!pip install -q fastapi uvicorn python-multipart faiss-gpu  


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00


In [7]:
!pip install -q -U datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00


In [8]:
# # Plant Disease Detection System - Core Implementation
# # ===================================================

# import os
# import torch
# import numpy as np
# from PIL import Image
# from ultralytics import YOLO
# from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer
# from datasets import Dataset
# import torchvision.transforms as transforms
# import json
# import yaml
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.llms import HuggingFaceHub
# from langchain.chains import RetrievalQA
# from langchain.prompts import PromptTemplate
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
# from langchain_community.llms import HuggingFacePipeline

# # ======================================
# # PART 1: YOLOv8 Training and Inference
# # ======================================

# def split_test_into_test_val(base_dir, split_ratio=0.5):
#     """
#     Splits the existing test directory into test and validation sets
#     while keeping the train directory intact.

#     Args:
#         base_dir: Base directory containing dataset
#         split_ratio: Ratio for splitting test data (0.5 means equal split)
#     """
#     import shutil

#     # Define paths
#     test_img_dir = os.path.join(base_dir, "test", "images")
#     test_lbl_dir = os.path.join(base_dir, "test", "labels")
#     val_img_dir = os.path.join(base_dir, "valid", "images")
#     val_lbl_dir = os.path.join(base_dir, "valid", "labels")

#     # Create validation directories if they don't exist
#     for dir_path in [val_img_dir, val_lbl_dir]:
#         if not os.path.exists(dir_path):
#             os.makedirs(dir_path)

#     # Get all image files from test directory
#     test_images = [f for f in os.listdir(test_img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

#     # Shuffle to ensure randomness
#     np.random.seed(42)  # For reproducibility
#     np.random.shuffle(test_images)

#     # Calculate split point for equal division
#     split_point = int(len(test_images) * split_ratio)

#     # Images to move to validation set
#     val_images = test_images[:split_point]

#     print(f"Splitting test data: {len(test_images)} images → {len(val_images)} val + {len(test_images) - len(val_images)} test")

#     # Move files to validation directory
#     moved_count = 0
#     for img in val_images:
#         # Move image
#         img_src_path = os.path.join(test_img_dir, img)
#         img_dst_path = os.path.join(val_img_dir, img)

#         # Get corresponding label file (same name, .txt extension)
#         label_file = os.path.splitext(img)[0] + '.txt'
#         lbl_src_path = os.path.join(test_lbl_dir, label_file)
#         lbl_dst_path = os.path.join(val_lbl_dir, label_file)

#         # Copy image and label to validation directory
#         if os.path.exists(img_src_path):
#             shutil.copy(img_src_path, img_dst_path)
#             os.remove(img_src_path)  # Remove from test folder

#             # Also move the label if it exists
#             if os.path.exists(lbl_src_path):
#                 shutil.copy(lbl_src_path, lbl_dst_path)
#                 os.remove(lbl_src_path)  # Remove from test labels folder
#                 moved_count += 1

#     print(f"Split complete. Moved {moved_count} images with labels to validation set.")
#     print(f"Validation set: {len(os.listdir(val_img_dir))} images")
#     print(f"Test set: {len(os.listdir(test_img_dir))} images")

#     # Update data.yaml file to include validation set
#     yaml_path = os.path.join(base_dir, "data.yaml")
#     if os.path.exists(yaml_path):
#         try:
#             with open(yaml_path, 'r') as f:
#                 yaml_data = yaml.safe_load(f)

#             # Add validation path if not present
#             if 'val' not in yaml_data:
#                 yaml_data['val'] = os.path.join(base_dir, "valid", "images")

#                 with open(yaml_path, 'w') as f:
#                     yaml.dump(yaml_data, f)

#                 print(f"Updated data.yaml with validation path")
#         except Exception as e:
#             print(f"Could not update data.yaml: {e}")

#     return {
#         'val': val_img_dir,
#         'test': test_img_dir
#     }

# def train_yolo_model(data_yaml_path, epochs=100, batch_size=16):
#     """
#     Train YOLOv8 model on custom plant disease dataset

#     Args:
#         data_yaml_path: Path to data.yaml file
#         epochs: Number of training epochs
#         batch_size: Batch size for training

#     Returns:
#         Path to trained model
#     """
#     print(f"Training YOLOv8 model with data config: {data_yaml_path}")

#     # Load a pre-trained model
#     model = YOLO("yolov8n.pt")  # Use YOLOv8 nano as base model

#     # Train the model
#     results = model.train(
#         data=data_yaml_path,
#         epochs=epochs,
#         imgsz=640,
#         batch=batch_size,
#         patience=15,  # Early stopping patience
#         save=True,    # Save best checkpoint
#         device=0 if torch.cuda.is_available() else 'cpu'
#     )

#     # Save the best model
#     model_path = "./yolov8_plant_disease.pt"
#     model.export(format="onnx")  # Export to ONNX format for deployment

#     print(f"YOLO model trained and saved to {model_path}")
#     return model_path

# # ======================================
# # PART 2: Vision Transformer (ViT) Fine-tuning
# # ======================================

# class PlantDiseaseDataset:
#     def __init__(self, image_dir, class_to_idx, transform=None):
#         """
#         Dataset for plant disease classification

#         Args:
#             image_dir: Directory containing images
#             class_to_idx: Dictionary mapping class names to indices
#             transform: Image transformations
#         """
#         self.image_dir = image_dir
#         self.transform = transform if transform else transforms.Compose([
#             transforms.Resize((224, 224)),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#         ])

#         self.class_to_idx = class_to_idx
#         self.imgs = self._make_dataset(image_dir, class_to_idx)

#         if len(self.imgs) == 0:
#             raise RuntimeError(f"Found 0 images in {image_dir}. Check your data directory structure.")

#         print(f"Loaded {len(self.imgs)} images for {len(class_to_idx)} classes")

#     def _make_dataset(self, image_dir, class_to_idx):
#         # First, check if image_dir has subdirectories (one per class)
#         subdirs = [d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]

#         instances = []

#         if subdirs and all(subdir in class_to_idx for subdir in subdirs):
#             # If organized by class subdirectories
#             for subdir in sorted(subdirs):
#                 if subdir in class_to_idx:
#                     target = class_to_idx[subdir]
#                     subdir_path = os.path.join(image_dir, subdir)
#                     for img_name in os.listdir(subdir_path):
#                         if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
#                             img_path = os.path.join(subdir_path, img_name)
#                             instances.append((img_path, target))
#         else:
#             # Try to infer class from filename (assuming format like "class_name_xxx.jpg")
#             # This approach is a fallback and might not be accurate for all datasets
#             for img_name in os.listdir(image_dir):
#                 if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
#                     found_class = False
#                     for class_name, idx in class_to_idx.items():
#                         # Class name should be part of the filename
#                         if class_name.lower() in img_name.lower():
#                             instances.append((os.path.join(image_dir, img_name), idx))
#                             found_class = True
#                             break

#                     # If no class match is found, assign to a default class (first class)
#                     if not found_class and class_to_idx:
#                         print(f"Warning: Could not determine class for {img_name}, assigning to default class")
#                         instances.append((os.path.join(image_dir, img_name), list(class_to_idx.values())[0]))

#         return instances

#     def __len__(self):
#         return len(self.imgs)

#     def __getitem__(self, idx):
#         img_path, target = self.imgs[idx]
#         image = Image.open(img_path).convert("RGB")

#         if self.transform:
#             image = self.transform(image)

#         return {
#             'pixel_values': image,
#             'labels': torch.tensor(target, dtype=torch.long)
#         }

# def create_class_mapping(class_names):
#     """Create a mapping from class names to indices"""
#     return {name: idx for idx, name in enumerate(class_names)}

# def finetune_vit(train_dir, val_dir, class_names, output_dir="./vit_model", batch_size=16, epochs=5):
#     """
#     Fine-tune ViT model on plant disease dataset
#     """
#     import os
    
#     # Disable Weights & Biases
#     os.environ['WANDB_DISABLED'] = 'true'
    
#     print("Fine-tuning ViT model for disease classification")
#     print(f"Train directory: {train_dir}")
#     print(f"Validation directory: {val_dir}")
#     print(f"Classes: {class_names}")

#     # Create directory for model
#     os.makedirs(output_dir, exist_ok=True)

#     # Create class mapping
#     class_to_idx = create_class_mapping(class_names)
#     id2label = {idx: label for label, idx in class_to_idx.items()}
#     label2id = {label: idx for idx, label in id2label.items()}

#     # Initialize feature extractor and model
#     model_name = "google/vit-base-patch16-224"
#     feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

#     # Load pretrained model
#     model = ViTForImageClassification.from_pretrained(
#         model_name,
#         num_labels=len(class_names),
#         id2label=id2label,
#         label2id=label2id,
#         ignore_mismatched_sizes=True
#     )

#     try:
#         # Create datasets
#         train_dataset = PlantDiseaseDataset(train_dir, class_to_idx)
#         val_dataset = PlantDiseaseDataset(val_dir, class_to_idx)

#         # Convert to datasets compatible with Hugging Face Trainer
#         def to_hf_dataset(custom_dataset):
#             data_dict = {
#                 'pixel_values': [],
#                 'labels': []
#             }

#             for i in range(len(custom_dataset)):
#                 try:
#                     item = custom_dataset[i]
#                     data_dict['pixel_values'].append(item['pixel_values'])
#                     data_dict['labels'].append(item['labels'])
#                 except Exception as e:
#                     print(f"Error processing item {i}: {e}")
#                     continue

#             return Dataset.from_dict(data_dict)

#         hf_train_dataset = to_hf_dataset(train_dataset)
#         hf_val_dataset = to_hf_dataset(val_dataset)

#         print(f"HF Train dataset size: {len(hf_train_dataset)} samples")
#         print(f"HF Validation dataset size: {len(hf_val_dataset)} samples")

#         # Set up training arguments
#         training_args = TrainingArguments(
#             output_dir=output_dir,
#             per_device_train_batch_size=batch_size,
#             per_device_eval_batch_size=batch_size,
#             evaluation_strategy="epoch",
#             save_strategy="epoch",
#             num_train_epochs=epochs,
#             learning_rate=5e-5,
#             load_best_model_at_end=True,
#             push_to_hub=False,
#             # Disable Weights & Biases
#             report_to='none'
#         )

#         # Set up trainer
#         def compute_metrics(eval_pred):
#             predictions, labels = eval_pred
#             predictions = np.argmax(predictions, axis=1)
#             return {"accuracy": (predictions == labels).mean()}

#         trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=hf_train_dataset,
#             eval_dataset=hf_val_dataset,
#             compute_metrics=compute_metrics,
#             tokenizer=feature_extractor,
#         )

#         # Train the model
#         trainer.train()

#         # Save model
#         model_path = os.path.join(output_dir, "best_model")
#         os.makedirs(model_path, exist_ok=True)

#         model.save_pretrained(model_path)
#         feature_extractor.save_pretrained(model_path)

#         print(f"ViT model fine-tuned and saved to {model_path}")
#         return model_path, feature_extractor

#     except Exception as e:
#         print(f"Error during ViT training: {e}")
#         # Fallback: Just save the pre-trained model without fine-tuning
#         model_path = os.path.join(output_dir, "pretrained_model")
#         os.makedirs(model_path, exist_ok=True)

#         model.save_pretrained(model_path)
#         feature_extractor.save_pretrained(model_path)

#         print(f"Could not fine-tune ViT model. Saved pretrained model to {model_path}")
#         return model_path, feature_extractor

# # ======================================
# # PART 3: Combined Detection Pipeline
# # ======================================

# class PlantDiseaseDetector:
#     def __init__(self, yolo_path, vit_path):
#         """
#         Combined disease detection and classification pipeline

#         Args:
#             yolo_path: Path to trained YOLO model
#             vit_path: Path to fine-tuned ViT model
#         """
#         print(f"Initializing detector with YOLO model: {yolo_path} and ViT model: {vit_path}")

#         # Load YOLO model
#         self.yolo_model = YOLO(yolo_path)

#         # Load fine-tuned ViT
#         self.vit_feature_extractor = ViTFeatureExtractor.from_pretrained(vit_path)
#         self.vit_model = ViTForImageClassification.from_pretrained(vit_path)
#         self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         self.vit_model.to(self.device)
#         self.vit_model.eval()

#     def detect_and_classify(self, image_path):
#         """
#         Detect and classify diseases in an image

#         Args:
#             image_path: Path to input image

#         Returns:
#             List of detected diseases with bounding boxes and confidence scores
#         """
#         print(f"Processing image: {image_path}")

#         # Step 1: Detect disease regions with YOLO
#         results = self.yolo_model(image_path)

#         # Step 2: For each detected region, classify with ViT
#         disease_predictions = []

#         # Get detected bounding boxes
#         boxes = results[0].boxes

#         if len(boxes) > 0:
#             original_image = Image.open(image_path).convert("RGB")

#             for box in boxes:
#                 # Extract bounding box coordinates
#                 x1, y1, x2, y2 = box.xyxy[0].tolist()
#                 x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

#                 # Get class and confidence from YOLO
#                 class_id = int(box.cls[0].item())
#                 yolo_class = self.yolo_model.names[class_id]
#                 yolo_conf = box.conf[0].item()

#                 # Crop the detected region
#                 cropped_region = original_image.crop((x1, y1, x2, y2))

#                 # Process for ViT
#                 inputs = self.vit_feature_extractor(images=cropped_region, return_tensors="pt")
#                 inputs = {k: v.to(self.device) for k, v in inputs.items()}

#                 with torch.no_grad():
#                     outputs = self.vit_model(**inputs)

#                 # Get prediction
#                 predicted_class_idx = outputs.logits.argmax(-1).item()
#                 vit_class = self.vit_model.config.id2label[predicted_class_idx]
#                 vit_conf = torch.softmax(outputs.logits, dim=1)[0, predicted_class_idx].item()

#                 # Use ViT classification as it's more precise, but keep YOLO detection
#                 disease_predictions.append({
#                     "disease": vit_class,
#                     "yolo_class": yolo_class,
#                     "confidence": vit_conf,
#                     "yolo_confidence": yolo_conf,
#                     "bbox": [x1, y1, x2, y2]
#                 })

#         print(f"Found {len(disease_predictions)} disease instances")
#         return disease_predictions

# # ======================================
# # PART 4: Vector Database for Knowledge
# # ======================================

# def create_disease_knowledge():
#     """
#     Create sample disease knowledge data

#     Returns:
#         List of disease information dictionaries
#     """
#     # This is sample data - in a real application, you would load this from a database or CSV
#     disease_data = [
#         {
#             "disease": "Apple Scab Leaf",
#             "symptoms": "Olive green to brown spots on leaves, often with a velvety texture. Severe infections cause leaf yellowing and premature leaf drop.",
#             "treatment": "Apply fungicides like captan or myclobutanil at 7-10 day intervals. Begin applications at green tip stage and continue until rainy periods end.",
#             "best_practices": "Choose resistant apple varieties, maintain good air circulation by proper pruning, remove fallen leaves in autumn to reduce spore sources.",
#             "source": "Cornell University Plant Disease Diagnostic Clinic"
#         },
#         {
#             "disease": "Apple rust leaf",
#             "symptoms": "Bright orange or yellow spots on the upper leaf surface, with tubular rust structures on the underside of leaves. Can cause premature leaf drop.",
#             "treatment": "Apply fungicides containing mancozeb or sulfur during the growing season, especially in spring. Remove nearby juniper plants (alternate host).",
#             "best_practices": "Plant resistant varieties, maintain good spacing between trees, avoid overhead irrigation, remove all cedar/juniper trees within 1km if possible.",
#             "source": "University of Minnesota Extension"
#         },
#         {
#             "disease": "Potato leaf early blight",
#             "symptoms": "Dark brown to black lesions with concentric rings giving a target-like appearance. Usually starts on lower/older leaves and moves upward.",
#             "treatment": "Apply fungicides containing chlorothalonil, copper, or mancozeb every 7-10 days when symptoms first appear. Ensure complete coverage.",
#             "best_practices": "Use certified disease-free seed potatoes, practice crop rotation (3-4 years), ensure proper plant spacing for air circulation, avoid overhead irrigation.",
#             "source": "American Phytopathological Society"
#         },
#         {
#             "disease": "Tomato leaf late blight",
#             "symptoms": "Large, dark brown blotches on leaves with white fungal growth on the undersides during humid conditions. Can spread rapidly during cool, wet weather.",
#             "treatment": "Apply copper-based fungicides or chlorothalonil preventatively before disease appears. Once identified, apply fungicides every 5-7 days.",
#             "best_practices": "Avoid overhead irrigation, provide good air circulation, remove and destroy infected plants immediately, use resistant varieties when available.",
#             "source": "Michigan State University Extension"
#         },
#         {
#             "disease": "Tomato leaf bacterial spot",
#             "symptoms": "Small, dark lesions on leaves, sometimes with a yellow halo. Lesions may coalesce causing leaf blight. Fruit can develop raised, scabby spots.",
#             "treatment": "Apply copper-based bactericides every 7-10 days during wet weather. Rotate with mancozeb products to reduce copper resistance.",
#             "best_practices": "Use pathogen-free seed and transplants, rotate crops for 2-3 years, avoid overhead irrigation, remove crop debris after harvest.",
#             "source": "Florida Department of Agriculture Extension Service"
#         },
#         {
#             "disease": "Tomato Septoria leaf spot",
#             "symptoms": "Small, circular spots with dark margins and gray or tan centers. Lower leaves are infected first, and severe infections can cause complete defoliation.",
#             "treatment": "Apply fungicides containing chlorothalonil, copper compounds, or mancozeb when symptoms first appear, continuing at 7-14 day intervals.",
#             "best_practices": "Remove infected leaves immediately, practice crop rotation, avoid overhead watering, use mulch to prevent soil splash onto leaves.",
#             "source": "Purdue University Plant and Pest Diagnostic Laboratory"
#         },
#         {
#             "disease": "grape leaf black rot",
#             "symptoms": "Small, reddish-brown spots on leaves that enlarge to lesions with tan centers and dark borders. Fruits develop circular lesions and eventually become mummified.",
#             "treatment": "Apply fungicides containing myclobutanil, thiophanate-methyl, or captan at 10-14 day intervals from early shoot growth until veraison.",
#             "best_practices": "Prune to improve air circulation, remove mummified fruits and infected leaves, control nearby wild grape vines, practice good sanitation.",
#             "source": "Ohio State University Extension"
#         }
#     ]

#     # Add more diseases from your class list
#     for disease_name in ['Tomato leaf', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf','Apple leaf','Peach leaf','Raspberry leaf', 'Soyabean leaf', 'Soybean leaf','Bell_pepper leaf spot', 'Bell_pepper leaf',"Corn Gray leaf spot", "Corn leaf blight", "Corn rust leaf", "Blueberry leaf", "Strawberry leaf", "Squash Powdery mildew leaf"]:
#         if any(d["disease"] == disease_name for d in disease_data):
#             continue

#         disease_data.append({
#             "disease": disease_name,
#             "symptoms": f"Various symptoms associated with {disease_name}.",
#             "treatment": f"Standard treatment recommendations for {disease_name}.",
#             "best_practices": f"Best agricultural practices to prevent {disease_name}.",
#             "source": "Agricultural Extension Service"
#         })

#     return disease_data

# def create_disease_vectordb():
#     """
#     Create vector database for disease knowledge

#     Returns:
#         FAISS vector database
#     """
#     print("Creating vector database for disease knowledge")

#     # Get disease data
#     disease_data = create_disease_knowledge()

#     # Save to JSON file for reference
#     with open("disease_knowledge.json", "w") as f:
#         json.dump(disease_data, f, indent=2)

#     # Create documents for vectorization
#     documents = []
#     for item in disease_data:
#         doc_text = f"""Disease: {item['disease']}
#         Symptoms: {item['symptoms']}
#         Treatment: {item['treatment']}
#         Best Practices: {item['best_practices']}
#         Source: {item['source']}"""
#         documents.append(doc_text)

#     # Split documents for better retrieval
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
#     split_docs = text_splitter.create_documents(documents)

#     # Use HuggingFace Embeddings
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#     # Create and save vector database
#     vectordb = FAISS.from_documents(split_docs, embeddings)
#     vectordb.save_local("disease_vectordb")

#     print(f"Vector database created with {len(split_docs)} document chunks")
#     return vectordb

# # ======================================
# # PART 5: RAG System with LangChain
# # ======================================

# class DiseaseRAGSystem:
#     def __init__(self, vectordb_path="disease_vectordb"):
#         """
#         RAG system for disease treatment recommendations
#         """
#         print(f"Initializing RAG system from {vectordb_path}")

#         # Load vector database
#         self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#         self.vectordb = FAISS.load_local(vectordb_path, self.embeddings, allow_dangerous_deserialization=True)

#         # Initialize a local model instead of HuggingFaceHub
#         model_id = "google/flan-t5-small"  # Using a smaller model that can run locally
#         tokenizer = AutoTokenizer.from_pretrained(model_id)
#         model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
        
#         # Create a text generation pipeline
#         pipe = pipeline(
#             "text2text-generation",
#             model=model, 
#             tokenizer=tokenizer, 
#             max_length=512
#         )
        
#         # Create LangChain LLM from the pipeline
#         self.llm = HuggingFacePipeline(pipeline=pipe)

#         # Create retriever
#         self.retriever = self.vectordb.as_retriever(search_kwargs={"k": 3})

#     def get_treatment_plan(self, disease_name):
#         """
#         Get treatment plan for a specific disease
#         Args:
#             disease_name: Name of the disease
#         Returns:
#             Dictionary with treatment plan and sources
#         """
#         print(f"Getting treatment plan for: {disease_name}")
        
#         try:
#             # Retrieve relevant documents
#             docs = self.retriever.get_relevant_documents(disease_name)
            
#             # If no documents found, return a generic response
#             if not docs:
#                 return {
#                     "disease": disease_name,
#                     "treatment_plan": f"No specific information found for {disease_name}. Consult a local agricultural expert.",
#                     "sources": []
#                 }
            
#             # Prepare context from retrieved documents
#             context = "\n\n".join([doc.page_content for doc in docs])
            
#             # Use the LLM to generate a treatment plan
#             prompt = f"""You are an agricultural expert. 
#             Provide a comprehensive treatment plan for {disease_name} based on the following information:

#             {context}

#             Treatment Plan for {disease_name}:"""
            
#             # Generate treatment plan
#             treatment_plan = self.llm(prompt)
            
#             return {
#                 "disease": disease_name,
#                 "treatment_plan": treatment_plan,
#                 "sources": [doc.page_content for doc in docs]
#             }
        
#         except Exception as e:
#             print(f"Error in get_treatment_plan for {disease_name}: {e}")
#             return {
#                 "disease": disease_name,
#                 "treatment_plan": f"Unable to generate treatment plan for {disease_name}. Please consult a local agricultural expert.",
#                 "sources": []
#             }

# # ======================================
# # PART 6: Main Execution
# # ======================================

# def check_dataset_structure(base_dir):
#     """
#     Check and print the structure of the dataset directory
#     """
#     print("\n=== Dataset Structure Check ===")

#     # Check train directory
#     train_img_dir = os.path.join(base_dir, "train", "images")
#     if os.path.exists(train_img_dir):
#         train_images = [f for f in os.listdir(train_img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
#         print(f"Train images: {len(train_images)} files")

#         # Print first few image names to see format
#         if train_images:
#             print(f"Sample train image names: {train_images[:5]}")
#     else:
#         print(f"Train directory not found at {train_img_dir}")

#     # Check data.yaml
#     yaml_path = os.path.join(base_dir, "data.yaml")
#     if os.path.exists(yaml_path):
#         try:
#             with open(yaml_path, 'r') as f:
#                 yaml_data = yaml.safe_load(f)
#             print(f"data.yaml contents: {yaml_data}")
#         except Exception as e:
#             print(f"Error reading data.yaml: {e}")
#     else:
#         print(f"data.yaml not found at {yaml_path}")

#     print("=== End of Dataset Check ===\n")


# # ======================================
# # PART 6: Main Execution
# # ======================================

# def main():
#     """
#     Main function to execute the entire pipeline
#     """
#     # Set paths
#     base_dir = "/kaggle/working/PlantDoc"
    
#     # Check dataset structure
#     check_dataset_structure(base_dir)
    
#     # Split test dataset
#     split_paths = split_test_into_test_val(base_dir, split_ratio=0.5)
#     data_yaml_path = os.path.join(base_dir, "data.yaml")
#     train_dir = os.path.join(base_dir, "train", "images")
#     val_dir = split_paths['val']
#     test_dir = split_paths['test']

#     # Load class names from data.yaml
#     with open(data_yaml_path, 'r') as f:
#         yaml_data = yaml.safe_load(f)
#         class_names = yaml_data.get('names', [])
    
#     # Validate class names
#     if not class_names:
#         print("Error: No class names found in data.yaml")
#         return

#     # Step 1: Load or train YOLO model
#     yolo_model_path = "/kaggle/input/yolo/pytorch/default/1/yolov8n (1).pt"
    
#     # Validate YOLO model path
#     if not os.path.exists(yolo_model_path):
#         print(f"Warning: YOLO model not found at {yolo_model_path}")
#         # Optional: Train a new model or use a different path
#         return

#     # Step 2: Fine-tune or load ViT model
#     try:
#         vit_model_path, _ = finetune_vit(train_dir, val_dir, class_names, epochs=3)
#     except Exception as e:
#         print(f"Error during ViT model preparation: {e}")
#         return

#     # Step 3: Create vector database
#     try:
#         vectordb = create_disease_vectordb()
#     except Exception as e:
#         print(f"Error creating vector database: {e}")
#         return

#     # Step 4: Initialize detector and RAG system
#     try:
#         detector = PlantDiseaseDetector(yolo_model_path, vit_model_path)
#         rag_system = DiseaseRAGSystem()
#     except Exception as e:
#         print(f"Error initializing detector or RAG system: {e}")
#         return

#     # Step 5: Test on sample images
#     test_images = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
#     if not test_images:
#         print("No test images found!")
#         return

#     for image_filename in test_images:
#         try:
#             sample_image_path = os.path.join(test_dir, image_filename)
            
#             # Detect and classify
#             predictions = detector.detect_and_classify(sample_image_path)

#             # Process predictions
#             if predictions:
#                 print(f"\nAnalyzing image: {image_filename}")
#                 for pred in predictions:
#                     try:
#                         treatment = rag_system.get_treatment_plan(pred["disease"])
#                         print(f"Disease: {pred['disease']}")
#                         print(f"Confidence: {pred['confidence']:.4f}")
#                         print(f"Treatment Plan:\n{treatment['treatment_plan']}")
#                     except Exception as pred_error:
#                         print(f"Error processing prediction: {pred_error}")
#             else:
#                 print(f"No diseases detected in {image_filename}")

#         except Exception as image_error:
#             print(f"Error processing image {image_filename}: {image_error}")

#     print("Crop disease detection system implementation completed!")

# if __name__ == "__main__":
#     main()

In [9]:
# !pip install -q transformers==4.30.0 torch sentence-transformers

In [10]:
!pip install -q transformers torch sentencepiece sentence-transformers

In [11]:
# !pip install -q httpcore==0.15.0 httpx==0.24.1 googletrans==4.0.0-rc1

In [12]:
!pip install -q deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00


In [13]:
# Plant Disease Detection System - Core Implementation
# ===================================================

import os
import re
import torch
import numpy as np
from PIL import Image
from ultralytics import YOLO
from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer
from datasets import Dataset
import torchvision.transforms as transforms
import json
import yaml
from googletrans.client import Translator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from deep_translator import GoogleTranslator, MyMemoryTranslator, LibreTranslator
import time

# ======================================
# PART 1: YOLOv8 Training and Inference
# ======================================

def split_test_into_test_val(base_dir, split_ratio=0.5):
    """
    Splits the existing test directory into test and validation sets
    while keeping the train directory intact.

    Args:
        base_dir: Base directory containing dataset
        split_ratio: Ratio for splitting test data (0.5 means equal split)
    """
    import shutil

    # Define paths
    test_img_dir = os.path.join(base_dir, "test", "images")
    test_lbl_dir = os.path.join(base_dir, "test", "labels")
    val_img_dir = os.path.join(base_dir, "valid", "images")
    val_lbl_dir = os.path.join(base_dir, "valid", "labels")

    # Create validation directories if they don't exist
    for dir_path in [val_img_dir, val_lbl_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    # Get all image files from test directory
    test_images = [f for f in os.listdir(test_img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    # Shuffle to ensure randomness
    np.random.seed(42)  # For reproducibility
    np.random.shuffle(test_images)

    # Calculate split point for equal division
    split_point = int(len(test_images) * split_ratio)

    # Images to move to validation set
    val_images = test_images[:split_point]

    print(f"Splitting test data: {len(test_images)} images → {len(val_images)} val + {len(test_images) - len(val_images)} test")

    # Move files to validation directory
    moved_count = 0
    for img in val_images:
        # Move image
        img_src_path = os.path.join(test_img_dir, img)
        img_dst_path = os.path.join(val_img_dir, img)

        # Get corresponding label file (same name, .txt extension)
        label_file = os.path.splitext(img)[0] + '.txt'
        lbl_src_path = os.path.join(test_lbl_dir, label_file)
        lbl_dst_path = os.path.join(val_lbl_dir, label_file)

        # Copy image and label to validation directory
        if os.path.exists(img_src_path):
            shutil.copy(img_src_path, img_dst_path)
            os.remove(img_src_path)  # Remove from test folder

            # Also move the label if it exists
            if os.path.exists(lbl_src_path):
                shutil.copy(lbl_src_path, lbl_dst_path)
                os.remove(lbl_src_path)  # Remove from test labels folder
                moved_count += 1

    print(f"Split complete. Moved {moved_count} images with labels to validation set.")
    print(f"Validation set: {len(os.listdir(val_img_dir))} images")
    print(f"Test set: {len(os.listdir(test_img_dir))} images")

    # Update data.yaml file to include validation set
    yaml_path = os.path.join(base_dir, "data.yaml")
    if os.path.exists(yaml_path):
        try:
            with open(yaml_path, 'r') as f:
                yaml_data = yaml.safe_load(f)

            # Add validation path if not present
            if 'val' not in yaml_data:
                yaml_data['val'] = os.path.join(base_dir, "valid", "images")

                with open(yaml_path, 'w') as f:
                    yaml.dump(yaml_data, f)

                print(f"Updated data.yaml with validation path")
        except Exception as e:
            print(f"Could not update data.yaml: {e}")

    return {
        'val': val_img_dir,
        'test': test_img_dir
    }

def train_yolo_model(data_yaml_path, epochs=100, batch_size=16):
    print(f"Training YOLOv8 model with data config: {data_yaml_path}")
    model = YOLO("yolov8n.pt")  # Start with pretrained weights
    
    # Add stronger augmentation parameters
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=640,
        batch=batch_size,
        patience=15,
        save=True,
        device=0 if torch.cuda.is_available() else 'cpu',
        # Key augmentation parameters
        hsv_h=0.1,  # Hue augmentation
        hsv_s=0.3,  # Saturation augmentation
        hsv_v=0.3,  # Value augmentation
        flipud=0.3, # Vertical flip
        fliplr=0.5, # Horizontal flip
        mosaic=0.5, # Mosaic augmentation probability
        degrees=10, # Rotation angle
        translate=0.1, # Translation
        scale=0.5,  # Scaling
        shear=0.0,  # Shear
        perspective=0.0 # Perspective transformation
    )
    
    # Get the path to the best saved model
    best_model_path = os.path.join(results.save_dir, 'weights', 'best.pt')
    
    # Export to ONNX format
    model.export(format="onnx")  # This will create best.onnx in the same directory
    
    print(f"YOLO model saved to: {best_model_path}")
    print(f"ONNX model exported to: {os.path.join(results.save_dir, 'weights', 'best.onnx')}")
    return best_model_path

# ======================================
# PART 2: Vision Transformer (ViT) Fine-tuning
# ======================================

class PlantDiseaseDataset:
    def __init__(self, image_dir, class_to_idx, transform=None):
        self.image_dir = image_dir
        self.class_to_idx = class_to_idx
        self.transform = transform if transform else transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(p=0.3),
            transforms.RandomRotation(20),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.imgs = self._make_dataset(image_dir, class_to_idx)
        if len(self.imgs) == 0:
            raise RuntimeError(f"Found 0 images in {image_dir}")

    def _make_dataset(self, image_dir, class_to_idx):
        instances = []
        # First check for organized directory structure
        for class_name in sorted(os.listdir(image_dir)):
            class_dir = os.path.join(image_dir, class_name)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                        img_path = os.path.join(class_dir, img_name)
                        instances.append((img_path, class_to_idx[class_name]))
        
        # If no organized dirs, try filename matching (fallback)
        if not instances:
            for img_name in os.listdir(image_dir):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    matched = False
                    for cls in class_to_idx:
                        if cls.lower() in img_name.lower():
                            instances.append((os.path.join(image_dir, img_name), class_to_idx[cls]))
                            matched = True
                            break
                    if not matched:
                        instances.append((os.path.join(image_dir, img_name), 0))  # Assign default class
        return instances

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path, target = self.imgs[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return {
            'pixel_values': image,
            'labels': torch.tensor(target, dtype=torch.long)
        }

def create_class_mapping(class_names):
    """Create a mapping from class names to indices"""
    return {name: idx for idx, name in enumerate(class_names)}

def finetune_vit(train_dir, val_dir, class_names, output_dir="./vit_model", batch_size=8, epochs=50):
    import os
    os.environ['WANDB_DISABLED'] = 'true'
    
    print("Fine-tuning ViT model for disease classification")
    os.makedirs(output_dir, exist_ok=True)
    
    # Create class mapping
    class_to_idx = create_class_mapping(class_names)
    id2label = {idx: label for label, idx in class_to_idx.items()}
    label2id = {label: idx for idx, label in id2label.items()}
    
    # Initialize feature extractor and model
    model_name = "google/vit-base-patch16-224"
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
    
    # Load pretrained model with dropout
    model = ViTForImageClassification.from_pretrained(
        model_name,
        num_labels=len(class_names),
        id2label=id2label,
        label2id=label2id,
        hidden_dropout_prob=0.3,
        attention_probs_dropout_prob=0.1,
        ignore_mismatched_sizes=True
    )
    
    try:
        # Create datasets
        train_dataset = PlantDiseaseDataset(train_dir, class_to_idx)
        val_dataset = PlantDiseaseDataset(val_dir, class_to_idx)
        
        # Check dataset sizes
        print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")
        if len(train_dataset) < 100 or len(val_dataset) < 30:
            raise ValueError("Insufficient data for training/validation")
        
        # # Check class distribution
        # def check_class_balance(dataset):
        #     labels = [item['labels'].item() for item in dataset]
        #     class_counts = {cls: labels.count(cls) for cls in set(labels)}
        #     print(f"Class distribution: {class_counts}")
        #     if min(class_counts.values()) < 5:
        #         print("Warning: Some classes are under-represented")
        
        # check_class_balance(train_dataset)
        # check_class_balance(val_dataset)
        
        # Convert to Hugging Face datasets
        def to_hf_dataset(custom_dataset):
            data_dict = {
                'pixel_values': [],
                'labels': []
            }
            for i in range(len(custom_dataset)):
                try:
                    item = custom_dataset[i]
                    data_dict['pixel_values'].append(item['pixel_values'])
                    data_dict['labels'].append(item['labels'])
                except Exception as e:
                    print(f"Error processing item {i}: {e}")
                    continue
            return Dataset.from_dict(data_dict)
        
        hf_train_dataset = to_hf_dataset(train_dataset)
        hf_val_dataset = to_hf_dataset(val_dataset)
        
        # Set up training arguments
        training_args = TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=epochs,
            learning_rate=2e-5,
            weight_decay=0.01,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            greater_is_better=True,
            early_stopping_patience=5,
            save_total_limit=2,
            report_to='none'
        )
        
        # Set up trainer
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, axis=1)
            return {"accuracy": (predictions == labels).mean()}
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=hf_train_dataset,
            eval_dataset=hf_val_dataset,
            compute_metrics=compute_metrics,
            tokenizer=feature_extractor,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
        )
        
        # Train the model
        trainer.train()
        
        # Save best model
        model_path = os.path.join(output_dir, "best_model")
        os.makedirs(model_path, exist_ok=True)
        model.save_pretrained(model_path)
        feature_extractor.save_pretrained(model_path)
        print(f"ViT model fine-tuned and saved to {model_path}")
        return model_path, feature_extractor
    
    except Exception as e:
        print(f"Error during ViT training: {e}")
        # Save pretrained model as fallback
        model_path = os.path.join(output_dir, "pretrained_model")
        os.makedirs(model_path, exist_ok=True)
        model.save_pretrained(model_path)
        feature_extractor.save_pretrained(model_path)
        print(f"Saved pretrained model to {model_path}")
        # ADD THIS LINE TO SAVE .pt WEIGHTS
        torch.save(model.state_dict(), os.path.join(model_path, "best_weights.pt"))
        return model_path, feature_extractor

# ======================================
# PART 3: Combined Detection Pipeline
# ======================================

class PlantDiseaseDetector:
    def __init__(self, yolo_path, vit_path):
        """
        Combined disease detection and classification pipeline

        Args:
            yolo_path: Path to trained YOLO model
            vit_path: Path to fine-tuned ViT model
        """
        print(f"Initializing detector with YOLO model: {yolo_path} and ViT model: {vit_path}")

        # Load YOLO model
        self.yolo_model = YOLO(yolo_path)

        # Load fine-tuned ViT
        self.vit_feature_extractor = ViTFeatureExtractor.from_pretrained(vit_path)
        self.vit_model = ViTForImageClassification.from_pretrained(vit_path)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.vit_model.to(self.device)
        self.vit_model.eval()

    def detect_and_classify(self, image_path):
        """
        Detect and classify diseases in an image

        Args:
            image_path: Path to input image

        Returns:
            List of detected diseases with bounding boxes and confidence scores
        """
        print(f"Processing image: {image_path}")

        # Step 1: Detect disease regions with YOLO
        results = self.yolo_model(image_path)

        # Step 2: For each detected region, classify with ViT
        disease_predictions = []

        # Get detected bounding boxes
        boxes = results[0].boxes

        if len(boxes) > 0:
            original_image = Image.open(image_path).convert("RGB")

            for box in boxes:
                # Extract bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

                # Get class and confidence from YOLO
                class_id = int(box.cls[0].item())
                yolo_class = self.yolo_model.names[class_id]
                yolo_conf = box.conf[0].item()

                # Crop the detected region
                cropped_region = original_image.crop((x1, y1, x2, y2))

                # Process for ViT
                inputs = self.vit_feature_extractor(images=cropped_region, return_tensors="pt")
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = self.vit_model(**inputs)

                # Get prediction
                predicted_class_idx = outputs.logits.argmax(-1).item()
                vit_class = self.vit_model.config.id2label[predicted_class_idx]
                vit_conf = torch.softmax(outputs.logits, dim=1)[0, predicted_class_idx].item()

                # Use ViT classification as it's more precise, but keep YOLO detection
                disease_predictions.append({
                    "disease": vit_class,
                    "yolo_class": yolo_class,
                    "confidence": vit_conf,
                    "yolo_confidence": yolo_conf,
                    "bbox": [x1, y1, x2, y2]
                })

        print(f"Found {len(disease_predictions)} disease instances")
        return disease_predictions

# ======================================
# PART 4: Vector Database for Knowledge
# ======================================

def create_disease_knowledge():
    """
    Create comprehensive disease knowledge data for plant diseases

    Returns:
        List of disease information dictionaries
    """
    # This is sample data - in a real application, you would load this from a database or CSV
    disease_data = [
        {
            "disease": "Apple Scab Leaf",
            "symptoms": "Olive green to brown spots on leaves, often with a velvety texture. Severe infections cause leaf yellowing and premature leaf drop.",
            "treatment": "Apply fungicides like captan or myclobutanil at 7-10 day intervals. Begin applications at green tip stage and continue until rainy periods end.",
            "best_practices": "Choose resistant apple varieties, maintain good air circulation by proper pruning, remove fallen leaves in autumn to reduce spore sources.",
            "source": "Cornell University Plant Disease Diagnostic Clinic"
        },
        {
            "disease": "Apple rust leaf",
            "symptoms": "Bright orange or yellow spots on the upper leaf surface, with tubular rust structures on the underside of leaves. Can cause premature leaf drop.",
            "treatment": "Apply fungicides containing mancozeb or sulfur during the growing season, especially in spring. Remove nearby juniper plants (alternate host).",
            "best_practices": "Plant resistant varieties, maintain good spacing between trees, avoid overhead irrigation, remove all cedar/juniper trees within 1km if possible.",
            "source": "University of Minnesota Extension"
        },
        {
            "disease": "Potato leaf early blight",
            "symptoms": "Dark brown to black lesions with concentric rings giving a target-like appearance. Usually starts on lower/older leaves and moves upward.",
            "treatment": "Apply fungicides containing chlorothalonil, copper, or mancozeb every 7-10 days when symptoms first appear. Ensure complete coverage.",
            "best_practices": "Use certified disease-free seed potatoes, practice crop rotation (3-4 years), ensure proper plant spacing for air circulation, avoid overhead irrigation.",
            "source": "American Phytopathological Society"
        },
        {
            "disease": "Tomato leaf late blight",
            "symptoms": "Large, dark brown blotches on leaves with white fungal growth on the undersides during humid conditions. Can spread rapidly during cool, wet weather.",
            "treatment": "Apply copper-based fungicides or chlorothalonil preventatively before disease appears. Once identified, apply fungicides every 5-7 days.",
            "best_practices": "Avoid overhead irrigation, provide good air circulation, remove and destroy infected plants immediately, use resistant varieties when available.",
            "source": "Michigan State University Extension"
        },
        {
            "disease": "Tomato leaf bacterial spot",
            "symptoms": "Small, dark lesions on leaves, sometimes with a yellow halo. Lesions may coalesce causing leaf blight. Fruit can develop raised, scabby spots.",
            "treatment": "Apply copper-based bactericides every 7-10 days during wet weather. Rotate with mancozeb products to reduce copper resistance.",
            "best_practices": "Use pathogen-free seed and transplants, rotate crops for 2-3 years, avoid overhead irrigation, remove crop debris after harvest.",
            "source": "Florida Department of Agriculture Extension Service"
        },
        {
            "disease": "Tomato Septoria leaf spot",
            "symptoms": "Small, circular spots with dark margins and gray or tan centers. Lower leaves are infected first, and severe infections can cause complete defoliation.",
            "treatment": "Apply fungicides containing chlorothalonil, copper compounds, or mancozeb when symptoms first appear, continuing at 7-14 day intervals.",
            "best_practices": "Remove infected leaves immediately, practice crop rotation, avoid overhead watering, use mulch to prevent soil splash onto leaves.",
            "source": "Purdue University Plant and Pest Diagnostic Laboratory"
        },
        {
            "disease": "grape leaf black rot",
            "symptoms": "Small, reddish-brown spots on leaves that enlarge to lesions with tan centers and dark borders. Fruits develop circular lesions and eventually become mummified.",
            "treatment": "Apply fungicides containing myclobutanil, thiophanate-methyl, or captan at 10-14 day intervals from early shoot growth until veraison.",
            "best_practices": "Prune to improve air circulation, remove mummified fruits and infected leaves, control nearby wild grape vines, practice good sanitation.",
            "source": "Ohio State University Extension"
        },
        {
            "disease": "Tomato leaf",
            "symptoms": "Healthy tomato leaves should be medium to dark green. Various diseases can cause yellowing, spotting, curling, or wilting of leaves.",
            "treatment": "Treatment depends on the specific condition affecting the tomato leaves. Monitor for early signs of disease and apply appropriate fungicides or bactericides.",
            "best_practices": "Provide adequate spacing between plants, use drip irrigation instead of overhead watering, rotate crops annually, and remove diseased plant material promptly.",
            "source": "University of California Integrated Pest Management Program"
        },
        {
            "disease": "Tomato mold leaf",
            "symptoms": "Yellow patches on upper leaf surfaces with corresponding gray-purple fuzzy growth on undersides. Leaves may curl, wither and die but remain attached to the stem.",
            "treatment": "Apply fungicides containing chlorothalonil, mancozeb, or copper at first sign of disease. Continue applications every 7-10 days in humid conditions.",
            "best_practices": "Increase plant spacing for better air circulation, prune lower leaves and suckers, avoid wetting foliage when watering, remove and destroy infected plant material.",
            "source": "Penn State Extension"
        },
        {
            "disease": "Tomato two spotted spider mites leaf",
            "symptoms": "Fine, light-colored stippling on upper leaf surfaces, yellowing or bronzing of leaves, fine webbing on undersides of leaves or between leaves and stems.",
            "treatment": "Apply miticides specifically labeled for spider mites. Insecticidal soaps or horticultural oils can be effective for light infestations.",
            "best_practices": "Regularly spray plants with water to disrupt mite colonies, introduce predatory mites, maintain plant vigor with proper watering and fertilization, avoid drought stress.",
            "source": "Texas A&M AgriLife Extension"
        },
        {
            "disease": "Apple leaf",
            "symptoms": "Healthy apple leaves should be medium to dark green. Watch for discoloration, spots, powdery substances, or curling which may indicate disease.",
            "treatment": "Regular monitoring and preventative fungicide applications based on local disease forecasting systems. Specific treatments depend on the particular disease.",
            "best_practices": "Select disease-resistant varieties, maintain tree vigor through proper nutrition, prune for good air circulation, practice orchard sanitation by removing fallen leaves.",
            "source": "Washington State University Tree Fruit Extension"
        },
        {
            "disease": "Peach leaf",
            "symptoms": "Healthy peach leaves are narrow and lance-shaped. Common issues include leaf curl (thickened, puckered, discolored leaves), bacterial spot (small angular spots), or powdery mildew (white powder).",
            "treatment": "For leaf curl, apply copper-based fungicides or synthetic fungicides during dormant season. For bacterial spot, apply copper-based bactericides before symptoms appear.",
            "best_practices": "Plant resistant varieties, maintain good air circulation, avoid overhead irrigation, remove and destroy infected leaves, apply dormant sprays.",
            "source": "Clemson Cooperative Extension"
        },
        {
            "disease": "Raspberry leaf",
            "symptoms": "Healthy raspberry leaves are green and serrated. Common issues include yellowing, spotting, or curling due to various viral, fungal, or bacterial infections.",
            "treatment": "Treatment depends on specific disease. May include fungicides for fungal diseases or removal of plants for viral diseases, which cannot be chemically treated.",
            "best_practices": "Purchase certified disease-free plants, control aphids and other insect vectors, maintain good air circulation, avoid overhead irrigation, remove wild brambles near plantings.",
            "source": "University of Maine Cooperative Extension"
        },
        {
            "disease": "Soyabean leaf",
            "symptoms": "Healthy soybean leaves are trifoliate and medium green. Watch for discoloration, spots, blotches, or defoliation which may indicate disease.",
            "treatment": "Apply appropriate fungicides based on disease identification. Timing is critical - applications should begin at first signs of disease or based on local disease forecasting.",
            "best_practices": "Rotate crops with non-host plants, plant resistant varieties, avoid working in fields when plants are wet, maintain proper fertility levels.",
            "source": "Iowa State University Extension"
        },
        {
            "disease": "Soybean leaf",
            "symptoms": "Frogeye leaf spot (circular gray spots with purple margins), brown spot (irregular brown spots), downy mildew (pale green to yellow spots on upper leaf surface, grayish tufts on lower surface).",
            "treatment": "Apply foliar fungicides containing strobilurin or triazole active ingredients at R1-R3 growth stages for most diseases. Follow resistance management practices.",
            "best_practices": "Select resistant varieties, practice crop rotation with non-host crops for 2-3 years, use high-quality seed, consider tillage to reduce inoculum levels.",
            "source": "University of Illinois Extension"
        },
        {
            "disease": "Bell_pepper leaf spot",
            "symptoms": "Small, circular water-soaked spots that enlarge and develop gray centers with dark borders. Severe infections cause leaf yellowing and premature leaf drop.",
            "treatment": "Apply copper-based fungicides every 7-10 days during wet weather. Rotate with other fungicides to prevent resistance development.",
            "best_practices": "Use pathogen-free seeds and transplants, practice crop rotation for 2-3 years, avoid overhead irrigation, remove and destroy infected plant debris.",
            "source": "University of Florida IFAS Extension"
        },
        {
            "disease": "Bell_pepper leaf",
            "symptoms": "Healthy bell pepper leaves are oval to lanceolate and dark green. Common issues include yellowing, spotting, curling, or wilting due to various pathogens.",
            "treatment": "Treatment depends on the specific condition. May include copper-based products for bacterial diseases or various fungicides for fungal diseases.",
            "best_practices": "Plant in well-drained soil, avoid overhead irrigation, space plants properly for good air circulation, rotate crops, remove diseased plant material.",
            "source": "New Mexico State University Extension"
        },
        {
            "disease": "Corn Gray leaf spot",
            "symptoms": "Rectangular, gray to tan lesions running parallel to leaf veins. Lesions may coalesce in severe infections, causing entire leaves to blight and die.",
            "treatment": "Apply foliar fungicides containing strobilurin, triazole, or mixed-mode of action products at tasseling or early silking stages when disease is present.",
            "best_practices": "Plant resistant hybrids, practice crop rotation with non-host crops for 1-2 years, use tillage to reduce surface residue when practical.",
            "source": "University of Nebraska-Lincoln Extension"
        },
        {
            "disease": "Corn leaf blight",
            "symptoms": "Long, elliptical, tan lesions with defined borders. Usually begins on lower leaves and moves upward. Severe infections can cause significant yield loss.",
            "treatment": "Apply foliar fungicides containing strobilurin, triazole, or mixed-mode of action products when disease is present on lower third of the plant prior to tasseling.",
            "best_practices": "Plant resistant hybrids, rotate crops with non-host plants, use tillage to bury infected residue, avoid over-irrigation.",
            "source": "Iowa State University Extension"
        },
        {
            "disease": "Corn rust leaf",
            "symptoms": "Small, circular to elongated, orange-brown pustules scattered across the leaf surface. Pustules rupture to release powdery spores that can be rubbed off with your finger.",
            "treatment": "Apply foliar fungicides containing triazoles or strobilurins when rust is identified in the area and environmental conditions favor disease development.",
            "best_practices": "Plant resistant hybrids, monitor fields regularly especially when conditions are favorable for rust development, time planting to avoid peak rust periods.",
            "source": "Purdue University Extension"
        },
        {
            "disease": "Blueberry leaf",
            "symptoms": "Healthy blueberry leaves are ovate and green. Common issues include red ringspot (circular red rings), mummy berry (brown V-shaped lesions), and rust (yellow spots on upper surface, orange pustules underneath).",
            "treatment": "Apply appropriate fungicides based on disease identification. Timing varies by disease, but often begins at budbreak and continues at 7-14 day intervals.",
            "best_practices": "Select resistant varieties, ensure good air circulation by proper pruning, remove wild blueberry plants from the vicinity, clean equipment between plantings.",
            "source": "Michigan State University Extension"
        },
        {
            "disease": "Strawberry leaf",
            "symptoms": "Healthy strawberry leaves are trifoliate and dark green. Common issues include leaf spot (purple to brown spots), leaf scorch (purple to brown spots without distinct centers), and powdery mildew (white powder).",
            "treatment": "Apply fungicides containing captan, myclobutanil, or copper compounds based on disease identification. Begin applications at first sign of disease.",
            "best_practices": "Use disease-free transplants, provide good air circulation, avoid overhead irrigation, remove old leaves after harvest, rotate plantings every 2-3 years.",
            "source": "University of California Small Farms Program"
        },
        {
            "disease": "Squash Powdery mildew leaf",
            "symptoms": "White, powdery fungal growth on upper and lower leaf surfaces. Begins as small, circular patches that expand to cover entire leaf. Severely infected leaves may yellow and die.",
            "treatment": "Apply fungicides containing sulfur, neem oil, potassium bicarbonate, or synthetic fungicides at first sign of disease. Continue applications at 7-14 day intervals.",
            "best_practices": "Plant resistant varieties, space plants for good air circulation, avoid overhead irrigation, remove and destroy infected leaves, rotate crops.",
            "source": "Cornell University Vegetable Program"
        }
    ]

    return disease_data

def create_disease_vectordb():
    """
    Create vector database for disease knowledge

    Returns:
        FAISS vector database
    """
    print("Creating vector database for disease knowledge")

    # Get disease data
    disease_data = create_disease_knowledge()

    # Save to JSON file for reference
    with open("disease_knowledge.json", "w") as f:
        json.dump(disease_data, f, indent=2)

    # Create documents for vectorization
    documents = []
    for item in disease_data:
        doc_text = f"""Disease: {item['disease']}
        Symptoms: {item['symptoms']}
        Treatment: {item['treatment']}
        Best Practices: {item['best_practices']}
        Source: {item['source']}"""
        documents.append(doc_text)

    # Split documents for better retrieval
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.create_documents(documents)

    # Use HuggingFace Embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create and save vector database
    vectordb = FAISS.from_documents(split_docs, embeddings)
    vectordb.save_local("disease_vectordb")

    print(f"Vector database created with {len(split_docs)} document chunks")
    return vectordb


# Bengali Translation Function
class RobustBengaliTranslator:
    def __init__(self):
        """
        Initialize the Bengali translator with multiple translation options
        using deep_translator library for more reliable translations
        """
        self.translation_cache = {}  # Cache to store previous translations
        
        # Bengali terms for common agricultural disease terminology
        self.default_translations = {
            # Common disease terms
            "leaf spot": "পাতার দাগ",
            "rust": "মরিচা",
            "blight": "ব্লাইট",
            "powdery mildew": "পাউডারি মিলডিউ",
            "downy mildew": "ডাউনি মিলডিউ",
            "wilt": "মুরঝে যাওয়া",
            "rot": "পচন",
            "mosaic": "মোজাইক",
            "yellowing": "হলুদ হয়ে যাওয়া",
            "bacterial": "ব্যাকটেরিয়াজনিত",
            "fungal": "ছত্রাকজনিত",
            "viral": "ভাইরাসজনিত",
            
            # Common symptom terms
            "discoloration": "বিবর্ণতা",
            "spots": "দাগ",
            "lesions": "ক্ষত",
            "wilting": "মুরঝে যাওয়া",
            "stunted growth": "বাধাগ্রস্ত বৃদ্ধি",
            
            # Common treatment terms
            "fungicide": "ছত্রাকনাশক",
            "pesticide": "কীটনাশক",
            "bactericide": "ব্যাকটেরিয়ানাশক",
            "spray": "স্প্রে",
            "remove infected plants": "আক্রান্ত গাছ অপসারণ করুন",
            "apple": "আপেল",
            "scab": "স্ক্যাব",
            "leaf": "পাতা"
        }
        
        # Initialize the translators
        self.translators = []
        
        # Try to initialize Google Translator
        try:
            self.google_translator = GoogleTranslator(source='en', target='bn')
            self.translators.append(("Google", self.google_translator))
            print("Google Translator initialized")
        except Exception as e:
            print(f"Failed to initialize Google Translator: {e}")
        
        # Try to initialize MyMemory Translator
        try:
            self.mymemory_translator = MyMemoryTranslator(source='en', target='bn')
            self.translators.append(("MyMemory", self.mymemory_translator))
            print("MyMemory Translator initialized")
        except Exception as e:
            print(f"Failed to initialize MyMemory Translator: {e}")
        
        # Try to initialize Libre Translator if you have a host
        try:
            # Note: You need to provide a host for LibreTranslator
            # self.libre_translator = LibreTranslator(source='en', target='bn', host='https://libretranslate.com/')
            # self.translators.append(("Libre", self.libre_translator))
            # print("Libre Translator initialized")
            pass
        except Exception as e:
            print(f"Failed to initialize Libre Translator: {e}")
            
        if not self.translators:
            print("WARNING: No reliable translators initialized. Using dictionary-based fallback only.")
        else:
            print(f"Successfully initialized {len(self.translators)} translators")

    def translate_to_bengali(self, text):
        """
        Translate English text to Bengali with multiple fallback options
        """
        if not text or text == "Information not available":
            return "তথ্য উপলব্ধ নয়"
            
        # Check cache first
        if text in self.translation_cache:
            return self.translation_cache[text]
            
        # Try to translate with available translators
        translation = None
        
        # Special case for disease names
        if len(text.split()) <= 4:  # Short text, likely a disease name
            # Try dictionary-based translation first for disease names
            dict_translation = self._dictionary_based_translation(text)
            if self._has_sufficient_bengali_content(dict_translation):
                translation = dict_translation
        
        # If not already translated, try each translator
        if not translation:
            for translator_name, translator in self.translators:
                try:
                    # Break text into chunks to handle API limitations
                    chunks = self._split_into_chunks(text)
                    translated_chunks = []
                    
                    for chunk in chunks:
                        # Add small delay to avoid rate limiting
                        time.sleep(0.5)
                        translated_chunk = translator.translate(chunk)
                        translated_chunks.append(translated_chunk)
                        
                    translation = ' '.join(translated_chunks)
                    print(f"Successfully translated with {translator_name}")
                    break
                except Exception as e:
                    print(f"{translator_name} translation failed: {e}")
                    continue
        
        # If all translators failed, use dictionary-based translation
        if not translation:
            translation = self._dictionary_based_translation(text)
            
        # Check if we have enough Bengali content
        if not self._has_sufficient_bengali_content(translation):
            # Post-process with dictionary to improve partial translations
            translation = self._enhance_partial_translation(translation)
        
        # Store in cache
        self.translation_cache[text] = translation
        return translation

    def _split_into_chunks(self, text, max_length=1000):
        """Split text into chunks to handle API limitations"""
        if len(text) <= max_length:
            return [text]
            
        chunks = []
        sentences = re.split(r'([.!?।])', text)
        current_chunk = ""
        
        for i in range(0, len(sentences), 2):
            if i+1 < len(sentences):
                sentence = sentences[i] + sentences[i+1]
            else:
                sentence = sentences[i]
                
            if len(current_chunk) + len(sentence) <= max_length:
                current_chunk += sentence
            else:
                chunks.append(current_chunk)
                current_chunk = sentence
                
        if current_chunk:
            chunks.append(current_chunk)
            
        return chunks

    def _dictionary_based_translation(self, text):
        """Use dictionary-based approach for translation"""
        result = text
        
        # Replace known phrases with translations
        for english, bengali in self.default_translations.items():
            # Case-insensitive replacement
            pattern = re.compile(r'\b' + re.escape(english) + r'\b', re.IGNORECASE)
            result = pattern.sub(bengali, result)
            
        return result

    def _enhance_partial_translation(self, text):
        """Apply dictionary translations to improve partial translations"""
        # First apply dictionary-based translations
        result = self._dictionary_based_translation(text)
        
        # Add Bengali markers for common sentence structures
        common_structures = {
            "should be": "করা উচিত",
            "can be": "করা যেতে পারে",
            "must be": "অবশ্যই করতে হবে",
            "is a": "একটি",
            "are": "হয়",
            "apply": "প্রয়োগ করুন",
            "remove": "অপসারণ করুন",
            "prevent": "প্রতিরোধ করুন",
            "treat": "চিকিৎসা করুন",
            "symptoms": "লক্ষণ",
            "treatment": "চিকিৎসা",
            "prevention": "প্রতিরোধ",
            "disease": "রোগ"
        }
        
        for english, bengali in common_structures.items():
            pattern = re.compile(r'\b' + re.escape(english) + r'\b', re.IGNORECASE)
            result = pattern.sub(bengali, result)
        
        return result

    def _has_sufficient_bengali_content(self, text):
        """Check if the text has enough Bengali content"""
        # Count Bengali characters
        bengali_chars = re.findall(r'[\u0980-\u09FF]', text)
        total_chars = len(text)
        
        # If we have at least 30% Bengali characters, consider it sufficient
        return len(bengali_chars) > 0.3 * total_chars
        
    def update_dictionary(self, english_term, bengali_translation):
        """Allow updating the translation dictionary at runtime"""
        if english_term and bengali_translation:
            self.default_translations[english_term.lower()] = bengali_translation
            # Clear cache entry if it exists
            if english_term in self.translation_cache:
                del self.translation_cache[english_term]
                
    def clear_cache(self):
        """Clear the translation cache"""
        self.translation_cache = {}
        print("Translation cache cleared")
        
    def translate_disease_name(self, name):
        """Special function to translate disease names"""
        # First try direct translation
        translated = self.translate_to_bengali(name)
        
        # Check if we have enough Bengali content
        if not self._has_sufficient_bengali_content(translated):
            # Special handling for disease names
            words = name.split()
            translated_words = []
            
            for word in words:
                word_lower = word.lower()
                if word_lower in self.default_translations:
                    translated_words.append(self.default_translations[word_lower])
                else:
                    # For untranslated disease names, keep original but add Bengali notation
                    translated_words.append(word)
                    
            translated = " ".join(translated_words)
            
            # Add Bengali suffix for disease names
            if not self._has_sufficient_bengali_content(translated):
                translated += f" ({name} এর বাংলা নাম)"
                
        return translated




# ======================================
# PART 5: RAG System with LangChain
# ======================================

class DiseaseRAGSystem:
    def __init__(self, vectordb_path="disease_vectordb"):
        """
        RAG system for disease treatment recommendations with Bengali translation
        """
        print(f"Initializing RAG system from {vectordb_path}")

        # Load vector database
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        self.vectordb = FAISS.load_local(vectordb_path, self.embeddings, allow_dangerous_deserialization=True)

        # Initialize Bengali Translator
        self.translator = RobustBengaliTranslator()

        try:
            # Initialize a local model for text generation
            from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
            model_id = "google/flan-t5-small"  # Using a smaller model that can run locally
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
            
            # Create a text generation pipeline
            pipe = pipeline(
                "text2text-generation",
                model=model, 
                tokenizer=tokenizer, 
                max_length=512
            )
            
            # Create LangChain LLM from the pipeline
            self.llm = HuggingFacePipeline(pipeline=pipe)
        except Exception as e:
            print(f"Error loading language model: {e}")
            # Define a simple fallback function that returns a generic response
            def fallback_llm(prompt):
                return {"generated_text": "Please apply appropriate fungicides and follow good agricultural practices."}
            self.llm = lambda prompt: fallback_llm(prompt)["generated_text"]

        # Create retriever
        self.retriever = self.vectordb.as_retriever(search_kwargs={"k": 3})

    def get_treatment_plan(self, disease_name):
        """Get treatment plan for a specific disease with better error handling and translation"""
        print(f"Getting treatment plan for: {disease_name}")
        
        try:
            # Direct lookup in knowledge base first
            disease_data = create_disease_knowledge()
            direct_match = None
            
            # Try to find exact match first
            for item in disease_data:
                if item["disease"].lower() == disease_name.lower():
                    direct_match = item
                    break
            
            # Try partial match if no exact match
            if not direct_match:
                for item in disease_data:
                    if disease_name.lower() in item["disease"].lower() or item["disease"].lower() in disease_name.lower():
                        direct_match = item
                        break
            
            # If we found a direct match, use it instead of RAG
            if direct_match:
                return self._create_response_from_direct_match(direct_match)
            
            # If no direct match, fall back to RAG approach
            docs = self.retriever.invoke(disease_name)
            
            if not docs:
                return self._create_fallback_response(disease_name)
                
            # Prepare context
            context = "\n".join([doc.page_content for doc in docs])
            
            # Extract information directly from retrieved documents
            symptoms = extract_section("Symptoms", context)
            treatment = extract_section("Treatment", context)
            best_practices = extract_section("Best Practices", context)
            
            # Use extracted info or fallbacks
            symptoms = symptoms if symptoms != "Information not available" else "Look for discoloration, spots, or abnormal growth on leaves."
            treatment = treatment if treatment != "Information not available" else "Apply appropriate fungicides according to local agricultural guidelines."
            prevention = best_practices if best_practices != "Information not available" else "Practice crop rotation, ensure good air circulation, and remove infected plant debris."
            
            # Use special disease name translation
            translated_disease = self.translator.translate_disease_name(disease_name)
            
            # Translate components with improved translator
            translated = {
                "disease": translated_disease,
                "symptoms": self.translator.translate_to_bengali(symptoms),
                "treatment": self.translator.translate_to_bengali(treatment),
                "prevention": self.translator.translate_to_bengali(prevention)
            }
            
            return {
                "english": {
                    "disease": disease_name,
                    "symptoms": symptoms,
                    "treatment": treatment,
                    "prevention": prevention
                },
                "bengali": translated,
                "sources": [doc.page_content[:100] + "..." for doc in docs]  # Truncated for display
            }
            
        except Exception as e:
            print(f"Error in treatment plan generation: {e}")
            return self._create_fallback_response(disease_name)

    def _create_response_from_direct_match(self, disease_item):
        """Create response using direct match from knowledge base"""
        # Translate components
        translated = {
            "disease": self.translator.translate_to_bengali(disease_item["disease"]),
            "symptoms": self.translator.translate_to_bengali(disease_item["symptoms"]),
            "treatment": self.translator.translate_to_bengali(disease_item["treatment"]),
            "prevention": self.translator.translate_to_bengali(disease_item["best_practices"])
        }
        
        return {
            "english": {
                "disease": disease_item["disease"],
                "symptoms": disease_item["symptoms"],
                "treatment": disease_item["treatment"],
                "prevention": disease_item["best_practices"]
            },
            "bengali": translated,
            "sources": [disease_item["source"]]
        }

    def _create_fallback_response(self, disease_name):
        """Create emergency response with basic translations"""
        generic_symptoms = "Discoloration, spots, wilting, or abnormal growth on plant tissue."
        generic_treatment = "Apply appropriate fungicides or bactericides. Remove infected plant parts. Improve plant nutrition."
        generic_prevention = "Ensure good air circulation, avoid overhead watering, practice crop rotation, and use disease-resistant varieties."
        
        translated = {
            "disease": self.translator.translate_to_bengali(disease_name),
            "symptoms": self.translator.translate_to_bengali(generic_symptoms),
            "treatment": self.translator.translate_to_bengali(generic_treatment),
            "prevention": self.translator.translate_to_bengali(generic_prevention)
        }
        
        return {
            "english": {
                "disease": disease_name,
                "symptoms": generic_symptoms,
                "treatment": generic_treatment,
                "prevention": generic_prevention
            },
            "bengali": translated,
            "sources": ["Generated fallback response"]
        }

# Helper functions
def extract_section(section_name, text):
    """Improved section extraction with better fallback handling"""
    try:
        # Try to find both "Section:" and "Section :" formats
        patterns = [
            rf"{section_name}:\s*(.*?)(?=\n\w+:|$)",
            rf"{section_name} :\s*(.*?)(?=\n\w+:|$)",
            rf"{section_name}(.*?)(?=\n\w+:|$)"  # Try without colon as last resort
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
            if matches and matches[0].strip():
                return matches[0].strip()
        
        # Check if the section_name exists in any form in the text
        lower_text = text.lower()
        if section_name.lower() in lower_text:
            # Find the start of the section
            start_idx = lower_text.find(section_name.lower())
            if start_idx != -1:
                # Get text after the section name until next section or end
                after_section = text[start_idx + len(section_name):]
                # Find the first 100 chars or until double newline
                end_idx = min(100, len(after_section))
                double_newline = after_section.find("\n\n")
                if double_newline != -1 and double_newline < end_idx:
                    end_idx = double_newline
                
                section_content = after_section[:end_idx].strip(": \n\t")
                if section_content:
                    return section_content
        
        # Direct keyword search as last resort
        keywords = {
            "Symptoms": ["spot", "lesion", "yellow", "brown", "discolor", "wilt"],
            "Treatment": ["apply", "spray", "fungicide", "remove", "prune", "treat"],
            "Prevention": ["rotation", "resistant", "spacing", "sanitation", "avoid"],
            "Best Practices": ["rotation", "resistant", "spacing", "sanitation", "avoid"]
        }
        
        if section_name in keywords:
            for keyword in keywords[section_name]:
                # Look for sentences containing the keyword
                sentences = re.findall(r'[^.!?]+[.!?]', text)
                for sentence in sentences:
                    if keyword.lower() in sentence.lower():
                        return sentence.strip()
        
        # If all attempts fail
        return "Information not available"
        
    except Exception as e:
        print(f"Error extracting section {section_name}: {e}")
        return "Information not available"

def _create_fallback_response(self, disease_name):
    """Create emergency response with basic translations"""
    bn_disease = self.translator.translate_to_bengali(disease_name)
    message = f"Apply appropriate fungicides and contact agricultural expert."
    
    return {
        "english": {
            "disease": disease_name,
            "message": message
        },
        "bengali": {
            "disease": bn_disease,
            "message": self.translator.translate_to_bengali(message)
        }
    }


# ======================================
# PART 6: Main Execution
# ======================================

def check_dataset_structure(base_dir):
    """
    Check and print the structure of the dataset directory
    """
    print("\n=== Dataset Structure Check ===")

    # Check train directory
    train_img_dir = os.path.join(base_dir, "train", "images")
    if os.path.exists(train_img_dir):
        train_images = [f for f in os.listdir(train_img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        print(f"Train images: {len(train_images)} files")

        # Print first few image names to see format
        if train_images:
            print(f"Sample train image names: {train_images[:5]}")
    else:
        print(f"Train directory not found at {train_img_dir}")

    # Check data.yaml
    yaml_path = os.path.join(base_dir, "data.yaml")
    if os.path.exists(yaml_path):
        try:
            with open(yaml_path, 'r') as f:
                yaml_data = yaml.safe_load(f)
            print(f"data.yaml contents: {yaml_data}")
        except Exception as e:
            print(f"Error reading data.yaml: {e}")
    else:
        print(f"data.yaml not found at {yaml_path}")

    print("=== End of Dataset Check ===\n")


# ======================================
# PART 6: Main Execution
# ======================================

def main():
    """
    Main function to execute the entire pipeline with Bengali translation using Google Translate
    """
    # Set paths
    base_dir = "/kaggle/working/PlantDoc"
    
    # Check dataset structure
    check_dataset_structure(base_dir)
    
    # Split test dataset
    split_paths = split_test_into_test_val(base_dir, split_ratio=0.5)
    data_yaml_path = os.path.join(base_dir, "data.yaml")
    train_dir = os.path.join(base_dir, "train", "images")
    val_dir = split_paths['val']
    test_dir = split_paths['test']

    # Load class names from data.yaml
    with open(data_yaml_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
        class_names = yaml_data.get('names', [])
    
    # Validate class names
    if not class_names:
        print("Error: No class names found in data.yaml")
        return

    # def check_class_distribution(data_dir):
    #         classes = os.listdir(data_dir)
    #         for cls in classes:
    #             img_count = len(os.listdir(os.path.join(data_dir, cls)))
    #             print(f"Class {cls}: {img_count} images")
    #             if img_count < 50:
    #                 print(f"Warning: Class {cls} has only {img_count} images. Consider data augmentation.")

    # # Add this check before training
    # check_class_distribution(train_dir)
    # check_class_distribution(val_dir)

    # Step 1: Load or train YOLO model
    yolo_model_path = "/kaggle/input/best/pytorch/default/1/best.pt"
    
    # Validate YOLO model path
    if not os.path.exists(yolo_model_path):
        print(f"Warning: YOLO model not found at {yolo_model_path}")
        return

    # Step 2: Fine-tune or load ViT model
    try:
        vit_model_path, _ = finetune_vit(train_dir, val_dir, class_names, epochs=5)
    except Exception as e:
        print(f"Error during ViT model preparation: {e}")
        return

    # Step 3: Create vector database
    try:
        vectordb = create_disease_vectordb()
    except Exception as e:
        print(f"Error creating vector database: {e}")
        return

    # Step 4: Initialize detector and RAG system
    try:
        detector = PlantDiseaseDetector(yolo_model_path, vit_model_path)
        rag_system = DiseaseRAGSystem()
    except Exception as e:
        print(f"Error initializing detector or RAG system: {e}")
        return

    # Step 5: Test on sample images
    test_images = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if not test_images:
        print("No test images found!")
        return

    for image_filename in test_images[:20]:  # Process first 5 images for testing
        try:
            sample_image_path = os.path.join(test_dir, image_filename)
            
            # Detect and classify
            predictions = detector.detect_and_classify(sample_image_path)

            # Process predictions
            if predictions:
                    print(f"\nAnalyzing image: {image_filename}")
                    for pred in predictions:
                        try:
                            treatment = rag_system.get_treatment_plan(pred["disease"])
                            
                            print("\n=== English ===")
                            print(f"Disease: {treatment['english']['disease']}")
                            print(f"Confidence: {pred['confidence']:.4f}")
                            print("\n🚩 Symptoms:")
                            print(treatment['english']['symptoms'])
                            print("\n💊 Treatment:")
                            print(treatment['english']['treatment'])
                            print("\n🛡️ Prevention:")
                            print(treatment['english']['prevention'])
                            
                            print("\n=== বাংলা (Bengali) ===")
                            print(f"রোগ: {treatment['bengali']['disease']}")
                            print("\n🚩 লক্ষণ:")
                            print(treatment['bengali']['symptoms'])
                            print("\n💊 চিকিৎসা:")
                            print(treatment['bengali']['treatment'])
                            print("\n🛡️ প্রতিরোধ:")
                            print(treatment['bengali']['prevention'])
                            
                        except Exception as pred_error:
                            print(f"Error processing prediction: {pred_error}")
        except Exception as image_error:
            print(f"Error processing image {image_filename}: {image_error}")

    print("Crop disease detection system implementation completed!")

if __name__ == "__main__":
    # Call the modified main function
     main()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

=== Dataset Structure Check ===
Train images: 2328 files
Sample train image names: ['cherry-fertilizer_jpg.rf.5055e585b666902f97a5e3e87b707864.jpg', 'late-blight-tomato_jpg.rf.88c5fd7d59668a3f7ebe8d5fb0bcec8e.jpg', '37310253806_04fee91918_b_jpg.rf.89e19f0964ab7e554560b8b0c2642949.jpg', 'autumn-red-apple-leaves_jpg.rf.b2d812dd805a4e7c779a348fa31fc316.jpg', 'p1020076cs_jpg.rf.065b0a5b4eed1fa0f3834a523c5ae341.jpg']
data.yaml contents: {'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 30, 'names': ['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf spot', 'Bell_pepper leaf', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot'

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-13-f45ed6f24c3c>:933: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  self.llm = HuggingFacePipeline(pipeline=pipe)


Processing image: /kaggle/working/PlantDoc/test/images/07c_jpg.rf.8ac68f057bc0959e2fc75afc530145df.jpg

image 1/1 /kaggle/working/PlantDoc/test/images/07c_jpg.rf.8ac68f057bc0959e2fc75afc530145df.jpg: 640x640 1 Corn Gray leaf spot, 6.5ms
Speed: 8.1ms preprocess, 6.5ms inference, 106.8ms postprocess per image at shape (1, 3, 640, 640)
Found 1 disease instances

Analyzing image: 07c_jpg.rf.8ac68f057bc0959e2fc75afc530145df.jpg
Getting treatment plan for: grape leaf black rot
Successfully translated with Google
Successfully translated with Google
Successfully translated with Google

=== English ===
Disease: grape leaf black rot
Confidence: 0.0753

🚩 Symptoms:
Small, reddish-brown spots on leaves that enlarge to lesions with tan centers and dark borders. Fruits develop circular lesions and eventually become mummified.

💊 Treatment:
Apply fungicides containing myclobutanil, thiophanate-methyl, or captan at 10-14 day intervals from early shoot growth until veraison.

🛡️ Prevention:
Prune to im